In [1]:
import sys
import subprocess

# Función para instalar paquetes si no están presentes
def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        print(f"📦 Instalando {package} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        globals()[package] = __import__(package)

# Instalar y cargar pandas y requests automáticamente
install_and_import("pandas")
install_and_import("requests")

print("✅ pandas y requests están listos")

✅ pandas y requests están listos


In [2]:
import requests
import pandas as pd
import time
import urllib.parse

API_KEY = "5c53834bd454afc0fe8162c93b0a6054"
URL = "http://ws.audioscrobbler.com/2.0/"
GENRES = ["jazz", "rap", "metal", "blues"]
LIMIT_ARTISTS = 500

cache = {}  # Para no pedir info repetida

def get_top_artists(genre, limit=LIMIT_ARTISTS):
    """Obtiene artistas de un género usando tag.gettopartists"""
    try:
        r = requests.get(URL, params={
            "method": "tag.gettopartists",
            "tag": genre,
            "limit": limit,
            "api_key": API_KEY,
            "format": "json"
        }, timeout=20).json()
        return r.get("topartists", {}).get("artist", [])
    except:
        return []

def get_artist_info(name):
    """Obtiene info extra de un artista (similares, bio, listeners, playcount)"""
    if name in cache:
        return cache[name]
    try:
        r = requests.get(URL, params={
            "method": "artist.getinfo",
            "artist": urllib.parse.quote(name),
            "api_key": API_KEY,
            "format": "json"
        }, timeout=20).json()
        a = r.get("artist", {})
        info = {
            "similares": ", ".join([x["name"] for x in a.get("similar", {}).get("artist", [])[:3]]),
            "bio": a.get("bio", {}).get("summary", ""),
            "listeners": a.get("stats", {}).get("listeners", ""),
            "playcount": a.get("stats", {}).get("playcount", "")
        }
    except:
        info = {"similares":"","bio":"","listeners":"","playcount":""}
    cache[name] = info
    return info

# ---------- Descargar artistas ----------
todos = []
años = list(range(2019, 2025))  # [2019, 2020, 2021, 2022, 2023, 2024]

for g in GENRES:
    print(f"\n🎵 Descargando artistas del género: {g} ...")
    top_artists = get_top_artists(g, limit=LIMIT_ARTISTS)
    
    for idx, a in enumerate(top_artists):
        nombre = a.get("name", "")
        if nombre:
            # Asignar años de forma cíclica (2019 → 2020 → … → 2024 → 2019 → …)
            anio = años[idx % len(años)]
            
            todos.append({
                "artista": nombre,
                "genero": g,
                "anio": anio,
                **get_artist_info(nombre)
            })
        time.sleep(0.1)

# ---------- Guardar CSV ----------
df = pd.DataFrame(todos)
df.to_csv("artistas_lastfm.csv", index=False, encoding="utf-8")

print("\n✅ CSV generado con columnas:", ", ".join(df.columns))
print("Total de artistas guardados:", len(df))



🎵 Descargando artistas del género: jazz ...

🎵 Descargando artistas del género: rap ...

🎵 Descargando artistas del género: metal ...

🎵 Descargando artistas del género: blues ...

✅ CSV generado con columnas: artista, genero, anio, similares, bio, listeners, playcount
Total de artistas guardados: 2000
